In [1]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

from Load_Predictions import *
from downstream_models import *
from Featurize import *
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
# from autogluon import TabularPrediction as task
import copy
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Numeric': 0,
# Categorical': 1,
# Datetime':2,
# Sentence':3,
# URL': 4,
# Numbers': 5,
# List': 6,
# Unusable': 7,
# Custom Object': 8

In [3]:
# Input from user

classification = 1
regression = 0
InputFilePath = 'datasets/supreme.csv'
TargetColumn = 'binaryClass'
TruthVector = [1,1,1,1,1,1,1] ### choose input from the dictionary above

In [4]:
file = InputFilePath
target_col = TargetColumn
dataDownstream = pd.read_csv(file)
dataDownstream

,Actions_taken,Liberal,Unconstitutional,Precedent_alteration,Unanimous,Year_of_decision,Lower_court_disagreement,binaryClass
0,0,1,0,0,1,1953,1,N
1,0,0,0,0,1,1953,0,N
2,0,0,0,0,0,1953,0,N
3,0,0,0,0,1,1953,1,N
4,0,1,0,0,0,1953,0,N
...,...,...,...,...,...,...,...,...
4047,0,0,0,0,0,1988,1,P
4048,0,1,1,0,0,1988,0,P
4049,0,1,0,0,0,1988,1,P
4050,0,0,0,0,0,1988,0,P


In [5]:
test_sizes = [1,2,3,4,5,10]

for sample_size in test_sizes:
    dataDownstream_sample = dataDownstream.sample(n=sample_size)
    y = dataDownstream_sample[[target_col]]
    dataDownstream_sample = dataDownstream_sample.drop(target_col, axis=1)

    dataFeaturized = FeaturizeFile(dataDownstream_sample)

    dataFeaturized1 = ProcessStats(dataFeaturized)
    dataFeaturized2 = FeatureExtraction(dataFeaturized,dataFeaturized1,0)
    dataFeaturized2 = dataFeaturized2.fillna(0)
    print(dataFeaturized2)

Column:  Actions_taken
Column:  Liberal
Column:  Unconstitutional
Column:  Precedent_alteration
Column:  Unanimous
Column:  Year_of_decision
Column:  Lower_court_disagreement
7 7
7 7 7 7
7
   total_vals  num_nans  %_nans  num_of_dist_val  %_dist_val    mean  std_dev  \
0           1         0     0.0                1       100.0     0.0      0.0   
1           1         0     0.0                1       100.0     1.0      0.0   
2           1         0     0.0                1       100.0     0.0      0.0   
3           1         0     0.0                1       100.0     0.0      0.0   
4           1         0     0.0                1       100.0     0.0      0.0   
5           1         0     0.0                1       100.0  1960.0      0.0   
6           1         0     0.0                1       100.0     0.0      0.0   

   min_val  max_val  has_delimiters  ...  1443  1444  1445  1446  1447  1448  \
0      0.0      0.0           False  ...     0     0     0     0     0     0   
1 

   total_vals  num_nans  %_nans  num_of_dist_val  %_dist_val    mean  \
0          10         0     0.0                2        20.0     0.4   
1          10         0     0.0                2        20.0     0.8   
2          10         0     0.0                2        20.0     0.5   
3          10         0     0.0                2        20.0     0.1   
4          10         0     0.0                2        20.0     0.4   
5          10         0     0.0                9        90.0  1971.7   
6          10         0     0.0                2        20.0     0.3   

    std_dev  min_val  max_val  has_delimiters  ...  1443  1444  1445  1446  \
0  1.200000      0.0      4.0           False  ...     0     0     0     0   
1  0.400000      0.0      1.0           False  ...     0     0     0     0   
2  0.500000      0.0      1.0           False  ...     0     0     0     0   
3  0.300000      0.0      1.0           False  ...     0     0     0     0   
4  0.489898      0.0      1.0    

In [6]:
y_RF = Load_RF(dataFeaturized2)
# y_pandas = Load_Pandas(dataDownstream)
# y_TFDV = Load_TFDV(dataDownstream)
# y_gl = Load_GLUON(dataDownstream , dataFeaturized)
y_truth = TruthVector

In [7]:
y_RF, y_truth

([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1, 1, 1, 1, 1, 1, 1])

In [8]:
y_cur_lst = [y_truth,y_RF,y_pandas,y_TFDV,y_gl]
for y_cur in y_cur_lst:
    all_cols = Featurize(dataDownstream,attribute_names,y_cur)
    print(all_cols)

    if classification:
        avgsc_train_lst_LR,avgsc_lst_LR,avgsc_hld_lst_LR = LogRegClassifier(all_cols,y)
        avgsc_train_lst_RF,avgsc_lst_RF,avgsc_hld_lst_RF = RandForestClassifier(all_cols,y)

    if regression:
        avgsc_train_lst_LR,avgsc_lst_LR,avgsc_hld_lst_LR = LinearRegression(all_cols,y)
        avgsc_train_lst_RF,avgsc_lst_RF,avgsc_hld_lst_RF = RandForestRegressor(all_cols,y)
        
    print('Linear Model:')
    print(avgsc_train_lst_LR)
    print(avgsc_lst_LR)
    print(avgsc_hld_lst_LR)

    print(np.mean(avgsc_train_lst_LR))
    print(np.mean(avgsc_lst_LR))
    print(np.mean(avgsc_hld_lst_LR))


    print('Random Forest:')
    print(avgsc_train_lst_RF)
    print(avgsc_lst_RF)
    print(avgsc_hld_lst_RF)

    print(np.mean(avgsc_train_lst_RF))
    print(np.mean(avgsc_lst_RF))
    print(np.mean(avgsc_hld_lst_RF))        

NameError: name 'y_pandas' is not defined

In [ ]:
# def FeaturizeFile(df):
#     # df = pd.read_csv(CSVfile,encoding = 'latin1')

#     stats = []
#     attribute_name = []
#     sample = []
#     id_value = []
#     i = 0

#     castability = []
#     number_extraction = []

#     avg_tokens = []
#     ratio_dist_val = []
#     ratio_nans = []

#     keys = list(df.keys())

#     attribute_name.extend(keys)
#     summary_stat_result = summary_stats(df, keys)
#     stats.extend(summary_stat_result)
#     samples = get_sample(df,keys)
#     sample.extend(samples)


#     # castability.extend(castability_feature(df, keys))
#     # number_extraction.extend(numeric_extraction(df, keys))

#     # avg_tokens.extend(get_avg_tokens(samples))
#     ratio_dist_val.extend(get_ratio_dist_val(summary_stat_result))
#     ratio_nans.extend(get_ratio_nans(summary_stat_result))


#     csv_names = ['Attribute_name', 'total_vals', 'num_nans', 'num_of_dist_val', 'mean', 'std_dev', 'min_val',
#                  'max_val', '%_dist_val', '%_nans', 'sample_1', 'sample_2', 'sample_3','sample_4','sample_5'
#                 ]
#     golden_data = pd.DataFrame(columns = csv_names)

#     for i in range(len(attribute_name)):
#         # print(attribute_name[i])
#         val_append = []
#         val_append.append(attribute_name[i])
#         val_append.extend(stats[i])
        
#         val_append.append(ratio_dist_val[i])
#         val_append.append(ratio_nans[i])    
        
#         val_append.extend(sample[i])
#     #     val_append.append(castability[i])
#     #     val_append.append(number_extraction[i])
#     #     val_append.append(avg_tokens[i])

#         golden_data.loc[i] = val_append
#     #     print(golden_data)


#     curdf = golden_data

#     for row in curdf.itertuples():

#         # print(row[11])
#         is_list = False
#         curlst = [row[11],row[12],row[13],row[14],row[15]]

#         delim_cnt,url_cnt,email_cnt,date_cnt =0,0,0,0
#         chars_totals,word_totals,stopwords,whitespaces,delims_count = [],[],[],[],[]

#         for value in curlst: 
#             word_totals.append(len(str(value).split(' ')))
#             chars_totals.append(len(str(value)))
#             whitespaces.append(str(value).count(' '))
   
#             if del_reg.match(str(value)):  delim_cnt += 1    
#             if url_reg.match(str(value)):  url_cnt += 1
#             if email_reg.match(str(value)):  email_cnt += 1

#             delims_count.append(len(delimeters.findall(str(value))))        

#             tokenized = word_tokenize(str(value))
#             # print(tokenized)
#             stopwords.append(len([w for w in tokenized if w in stop_words]))    

#             try:
#                 _ = pd.Timestamp(value)
#                 date_cnt += 1
#             except ValueError: date_cnt += 0    

#         # print(delim_cnt,url_cnt,email_cnt)
#         if delim_cnt > 2:  curdf.at[row.Index, 'has_delimiters'] = True
#         else: curdf.at[row.Index, 'has_delimiters'] = False

#         if url_cnt > 2:  curdf.at[row.Index, 'has_url'] = True
#         else: curdf.at[row.Index, 'has_url'] = False
 
#         if email_cnt > 2:  curdf.at[row.Index, 'has_email'] = True
#         else: curdf.at[row.Index, 'has_email'] = False   

#         if date_cnt > 2:  curdf.at[row.Index, 'has_date'] = True
#         else: curdf.at[row.Index, 'has_date'] = False           

#         curdf.at[row.Index, 'mean_word_count'] = np.mean(word_totals)
#         curdf.at[row.Index, 'std_dev_word_count'] = np.std(word_totals)

#         curdf.at[row.Index, 'mean_stopword_total'] = np.mean(stopwords)
#         curdf.at[row.Index, 'stdev_stopword_total'] = np.std(stopwords)

#         curdf.at[row.Index, 'mean_char_count'] = np.mean(chars_totals)    
#         curdf.at[row.Index, 'stdev_char_count'] = np.std(chars_totals)

#         curdf.at[row.Index, 'mean_whitespace_count'] = np.mean(whitespaces)
#         curdf.at[row.Index, 'stdev_whitespace_count'] = np.std(whitespaces)    

#         curdf.at[row.Index, 'mean_delim_count'] = np.mean(whitespaces)
#         curdf.at[row.Index, 'stdev_delim_count'] = np.std(whitespaces)      

#         if curdf.at[row.Index, 'has_delimiters'] and curdf.at[row.Index, 'mean_char_count'] < 100: curdf.at[row.Index, 'is_list'] = True    
#         else: curdf.at[row.Index, 'is_list'] = False

#         if curdf.at[row.Index, 'mean_word_count'] > 10: curdf.at[row.Index, 'is_long_sentence'] = True    
#         else: curdf.at[row.Index, 'is_long_sentence'] = False    

#         # print(np.mean(stopwords))
#         # print('\n\n\n')

#     golden_data = curdf
#     return golden_data